##### Copyright &copy; 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Pipeline Tutorial using Penguin dataset

***A Short tutorial to run a simple TFX pipeline.***

Note: We recommend running this tutorial in a Colab notebook, with no setup required!  Just click "Run in Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple">
<img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/penguin_simple.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/penguin_simple.ipynb">
<img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
</table></div>

This Colab-based tutorial will create and run a TFX pipeline. The pipeline will consist of three essential components, ExampleGen, Trainer and Pusher. Please refer other tutorials like [Component tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) to learn full-capability of TFX.

## Setup
First, we install and import the necessary packages, set up paths, and download data.

### Upgrade Pip

To avoid upgrading Pip in a system when running locally, check to make sure that we're running in Colab.  Local systems can of course be upgraded separately.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Install TFX

**Note: In Google Colab, because of package updates, the first time you run this cell you must restart the runtime (Runtime > Restart runtime ...).**

In [ ]:
!pip install -q -U --use-feature=2020-resolver --pre tfx

#### Did you restart the runtime?

If you are using Google Colab, the first time that you run the cell above, you must restart the runtime (Runtime > Restart runtime ...). This is because of the way that Colab loads packages.

### Import packages
We import necessary packages, including standard TFX component classes.

In [ ]:
import os
import urllib

from absl import logging
import tensorflow as tf
tf.get_logger().propagate = False

import tfx

Let's check the library versions.

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

### Set up pipeline paths

In [ ]:
_pipeline_name = 'penguin_simple'

# Path to various pipeline artifact.
_pipeline_root = os.path.join('pipelines', _pipeline_name)
# Path to ML metadata Sqlite db.
_metadata_path = os.path.join('metadata', _pipeline_name,
                              'metadata.db')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join('serving_model', _pipeline_name)

# Set up logging.
logging.set_verbosity(logging.INFO)

### Download example data
We download the example dataset for use in our TFX pipeline. The dataset we're using is [Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) which is also used in the [examples](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin). 

There are four numeric features in this dataset:

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

All features were already normalized to 0~1. We will build a classification model which predicts the `species` of penguins.


In [ ]:
_data_root = os.path.join('data', _pipeline_name)
os.makedirs(_data_root)
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/penguins_processed.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

Take a quick look at the CSV file.

In [ ]:
!head {_data_filepath}

## Create a pipeline

TFX pipelines are defined using Python APIs. We will define a pipeline which consists of following three components.
- CsvExampleGen: Reads in data files and convert them to TFX internal format for further processing. There are multiple [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen)s for various formats. In this tutorial, we will use CsvExampleGen which takes CSV file input.
- Trainer: Trains a ML model. [Trainer component](https://www.tensorflow.org/tfx/guide/trainer) requires a model definition code from users. You can use TensorFlow APIs to specify how to train a model and export it in a saved_model format.
- Pusher: Copies the trained model outside of the TFX pipeline. [Pusher component](https://www.tensorflow.org/tfx/guide/pusher) can be thought of an deployment process of the trained ML model.

Before actually define the pipeline, we need to write a model code for the Trainer component first.

### Write model code.

We will create a simple DNN model for classification using TensorFlow Keras API. This model code will be saved to a separate file.


In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List, Text
import absl
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

# The Penguin dataset has 342 records, and is divided into train and eval
# splits in a 2:1 ratio.
_TRAIN_DATA_SIZE = 228
_EVAL_DATA_SIZE = 114
_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

_FEATURE_SPEC = {
    'culmen_length_mm': tf.io.FixedLenFeature(shape=[1], dtype=tf.float32),
    'culmen_depth_mm': tf.io.FixedLenFeature(shape=[1], dtype=tf.float32),
    'flipper_length_mm': tf.io.FixedLenFeature(shape=[1], dtype=tf.float32),
    'body_mass_g': tf.io.FixedLenFeature(shape=[1], dtype=tf.float32),
    'species': tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _get_serve_tf_examples_fn(model, feature_spec):
  """Returns a function that parses a serialized tf.Example."""

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

    return model(parsed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema)


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3, activation='softmax')(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss='sparse_categorical_crossentropy',
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=absl.logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  mirrored_strategy = tf.distribute.MirroredStrategy()
  with mirrored_strategy.scope():
    model = _build_keras_model()

  steps_per_epoch = _TRAIN_DATA_SIZE // _TRAIN_BATCH_SIZE

  # Write logs to path
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      epochs=fn_args.train_steps // steps_per_epoch,
      steps_per_epoch=steps_per_epoch,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model, _FEATURE_SPEC).get_concrete_function(
              tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

### Write a pipeline definition

We define a function to create a TFX pipeline. A `Pipeline` object represents a TFX pipeline which can be run using one of pipeline orchestration system that TFX supports.

In [ ]:
from typing import List, Text

from tfx.components import CsvExampleGen
from tfx.components import Pusher
from tfx.components import Trainer
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.components.base import executor_spec
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input

def _create_pipeline(pipeline_name: Text, pipeline_root: Text, data_root: Text,
                     module_file: Text, serving_model_dir: Text,
                     metadata_path: Text) -> pipeline.Pipeline:
  """Implements the penguin pipeline with TFX."""
  examples = external_input(data_root)

  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = CsvExampleGen(input=examples)

  # Uses user-provided Python function that trains a model.
  trainer = Trainer(
      module_file=module_file,
      custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
      examples=example_gen.outputs['examples'],
      train_args=trainer_pb2.TrainArgs(num_steps=100),
      eval_args=trainer_pb2.EvalArgs(num_steps=5))

  # Pushes the model to a file destination if check passed.
  pusher = Pusher(
      model=trainer.outputs['model'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components,
      enable_cache=True,
      metadata_connection_config=metadata.sqlite_metadata_connection_config(
          metadata_path))

### Run the pipeline using LocalDagRunner

We will use `LocalDagRunner`. LocalDagRunner is a simple local orchestrator without any external dependency and it is good for fast iteration during pipeline development or debugging.


In [ ]:
from tfx.orchestration.local import local_dag_runner
local_dag_runner.LocalDagRunner().run(
    _create_pipeline(
        pipeline_name=_pipeline_name,
        pipeline_root=_pipeline_root,
        data_root=_data_root,
        module_file=_trainer_module_file,
        serving_model_dir=_serving_model_dir,
        metadata_path=_metadata_path))

If the pipeline ran successfully, you should see "Component Pusher is finished." in the log.

You can find the generated model at the `serving_model`(the value of `_serving_model_dir` variable) if you didn't change the above code. There should be other artifacts under `pipelines`(the value of `_pipeline_root` variable). If you are on Colab, click folder icon on the left sidebar to browse files.


## Next steps

TFX provides many useful components and you can also create your own easily. Please see [TFX components tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras). If you want to deploy your pipeline into Google Cloud, please see [TFX on Cloud AI Platform](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines).

You can find more resources on https://www.tensorflow.org/tfx/tutorials.
